**<h1 align="center">PROYECTO INDIVIDUAL 1</h1>**
<h1 align="center"> (Machine Learning)</h1>

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

<h2>Preparacion de datos para similitud de coseno con variables que mas se correlacionan</h2>

In [6]:
#Cargamos los datos
df = pd.read_parquet('Merge\\Tabla_API.parquet')
df1 = pd.read_parquet('Merge\\Tabla_API.parquet')
#Eliminamos columnas que no se utilizaran
df.drop(['genres', 'review', 'playtime_forever', 'recommend', 'year'], axis=1, inplace=True)
df1.drop(['genres', 'review', 'playtime_forever', 'recommend', 'year'], axis=1, inplace=True)
#Cambiamos el tipo de variable
df['item_id'] = df['item_id'].astype(str)
df['year_posted'] = df['year_posted'].astype(str)
#Eliminamos valores duplicados
df = df.drop_duplicates(subset='item_id', keep='first')
df1 = df1.drop_duplicates(subset='item_id', keep='first')
#Combinamos los valores de las columnas "title, year_posted, developer" en una sola columna
df['features'] = df[['title', 'year_posted', 'developer']].apply(lambda x: ', '.join(x), axis=1)
df.drop(['title','year_posted','developer'],axis=1,inplace=True)
#Exportamos los datasets limpios de juegos
df.to_csv('DatasetsLimpios\\games_steam.csv', index=False)
df1.to_csv('DatasetsLimpios\\games_id.csv', index=False)

<h2>Definicion de funcion de modelo Machine Learning</h2>

In [7]:
def recomendacion_juego(item_id):
    #Cargar datos para utilizar en dos Dataframes distintos
    df = pd.read_csv('DatasetsLimpios\\games_steam.csv')
    df1 = pd.read_csv('DatasetsLimpios\\games_id.csv')

    #Crear una matriz de juegos
    tfidv = TfidfVectorizer(min_df=2, max_df=0.7, token_pattern=r'\b[a-zA-Z0-9]\w+\b')
    data_vector = tfidv.fit_transform(df['features'])

    data_vector_df = pd.DataFrame(data_vector.toarray(), index=df['item_id'], columns = tfidv.get_feature_names_out())

    #Calcular la similitud del coseno entre los juegos en la matriz de juegos
    vector_similitud_coseno = cosine_similarity(data_vector_df.values)

    cos_sim_df = pd.DataFrame(vector_similitud_coseno, index=data_vector_df.index, columns=data_vector_df.index)

    juego_simil = cos_sim_df.loc[item_id]

    simil_ordenada = juego_simil.sort_values(ascending=False)
    resultado = simil_ordenada.head(6).reset_index()

    result_df = resultado.merge(df1, on='item_id',how='left')

    #Devolver una lista con los 6 juegos mas similares
    juego_title = df1[df1['item_id'] == item_id]['title'].values[0]

    #Devuelve un mensaje que indica el juego original y los juegos recomendados
    mensaje = f"Si te gustó el juego {item_id} : {juego_title}, también te pueden gustar:"

    result_dict = {
        'mensaje': mensaje,
        'juegos recomendados': result_df['title'][1:6].tolist()
    }

    return result_dict

In [8]:
ab = recomendacion_juego(22200)
ab

{'mensaje': 'Si te gustó el juego 22200 : Zeno Clash, también te pueden gustar:',
 'juegos recomendados': ['Zeno Clash 2',
  'Rock of Ages',
  'Clash of the Monsters',
  'Abyss Odyssey',
  'NAM']}